# 3.3. Choropleth map vs. Points on map (scatter geo)

Prepare

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
import json
import urllib.request 

In [ ]:
def layout_helper(fig):
    fig.update_traces(marker_line_width=0.1)
    return fig

In [ ]:
with urllib.request.urlopen(
    "https://thomann-public.s3.eu-west-1.amazonaws.com/jst-mapviz/gmd_shape_int.geojson") as url:
    communities = json.loads(url.read().decode())

As we will build the same type of visualisation over and over again, let's put this into a function.

In [ ]:
def map_viz(df, geodata, geocode_df, geocode_json, colorvar):
    fig = px.choropleth_mapbox(df, geojson=geodata,
                           locations=df[geocode_df],
                           featureidkey="properties.{}".format(geocode_json),
                           color=colorvar,
                           mapbox_style="white-bg",
                           zoom=6.3, 
                           center = {"lat": 46.8, "lon": 8.5},
                           width=900,height=500
                           )
    layout_helper(fig).show()

We will load a wider dataset that contains more information for our communities.
* population_total
* population_foreigners
* population_age_20_to_39
* population_ratio_foreigners
*population_ratio_age_20_to_39
*taxable_income_chf
*taxable_income_per_capita
*cinema_number
*cinema_hall_number
*cinema_seats_number
*vacancy_rate
*car_number_total
*car_number_electric_hybrid
*car_ratio_electric_hybrid
*population_immigrants_this_year_20_to_39
*population_immigrants_this_year
*party_strength_svp
*party_strength_sp
*party_strength_gps
*party_strength_fdp
*party_strength_glp
*party_strength_cvp
*party_strength_bdp
*party_strength_evp
*party_strength_other
*turnout

In [ ]:
df_bfs = pd.read_csv('https://thomann-public.s3.eu-west-1.amazonaws.com/jst-mapviz/bfs_communities.csv')

Let's take for instance the number of cinemas.
Visualising this on a choropleth map as before looks a bit odd, and it doesn't really tell us quickly at one glance which city has the most cinemas.

In [ ]:
map_viz(df_bfs,communities,'community_code','GMDNR','cinema_number')

Instead, we want to visualise this as **points on a map** (scatter on map).
For this, we need to load centroids of the cantons and we need to add this information to our dataframe.

In [ ]:
with urllib.request.urlopen(
    "https://thomann-public.s3.eu-west-1.amazonaws.com/jst-mapviz/gmd_point_int.geojson") as url:
    communities_point = json.loads(url.read().decode())

In [ ]:
com = pd.json_normalize(communities_point['features'])

In [ ]:
com = com.merge(df_bfs, left_on='properties.GMDNR',right_on='community_code',how='left')

In [ ]:
def split_coordinates(coordinate, returnvalue):
    long = coordinate[0]
    lat = coordinate[1]
    if returnvalue=='lat':
        return lat
    else:
        return long


In [ ]:
com['latitude'] = com['geometry.coordinates'].apply(split_coordinates,returnvalue='lat')
com['longitude'] = com['geometry.coordinates'].apply(split_coordinates,returnvalue='long')

In [ ]:
fig = px.scatter_mapbox(com_cinema, lat="latitude", lon="longitude", size="cinema_number",
                  size_max=20, zoom=6.3, mapbox_style="carto-positron",width=800,height=500)
fig.show()

Also, using scatter on map, we can also visualise two attributes at once (one for size, one for color). With choropleth maps, you are limited to showing just one attribute.

In [ ]:
fig = px.scatter_mapbox(com_cinema, lat="latitude", lon="longitude", size="cinema_number",color='cinema_seats_number',
                  size_max=20, zoom=6.3, mapbox_style="carto-positron",width=800,height=500,
                        color_continuous_scale=px.colors.sequential.matter)
fig.show()

### Exercise
Visualise some more variables that you find interesting from df_bfs using either choropleth or scatter on map.